## Acceso a Drive

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.9 MB/s eta 0:00:00


In [ ]:
import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

In [ ]:
!pip install transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00


In [ ]:
pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skmultilearn.problem_transform import BinaryRelevance

In [ ]:
# Función de limpieza de texto
def clean_text(text):
    pattern = r'https://t.co/[^\s]+|@[a-zA-Z0-9_]{1,15}|[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+|[^\w\s]'
    clean_text = re.sub(pattern, '', text)
    clean_text = clean_text.lower()
    return clean_text

# Cargar datos
text_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/NLPFundamentals/Contest/sources/csvs/train_data.csv", names=['image', 'text'],header=None)
labels_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/NLPFundamentals/Contest/sources/csvs/train_labels_subtask_2.csv", names=['accidente', 'asesinato', 'robo', 'otro'],header=None)

d = {'image': text_df['image'].values, 'text': text_df['text'].values, 'accidente': labels_df['accidente'].values, 'asesinato': labels_df['asesinato'].values, 'robo': labels_df['robo'].values, 'otro': labels_df['otro'].values}
df = pd.DataFrame(data=d)

# Limpieza de datos
df_Preprocessing = df.copy()
df_Preprocessing.drop('image', axis=1, inplace=True)
df_Preprocessing['text'] = df_Preprocessing['text'].apply(clean_text)
df_Preprocessing['accidente'] = df_Preprocessing['accidente'].astype(float)
df_Preprocessing['asesinato'] = df_Preprocessing['asesinato'].astype(float)
df_Preprocessing['robo'] = df_Preprocessing['robo'].astype(float)
df_Preprocessing['otro'] = df_Preprocessing['otro'].astype(float)

# Divide el dataset
xFeatures = df_Preprocessing['text']
yLabels = df_Preprocessing[['accidente', 'asesinato', 'robo', 'otro']]
x_train, x_test, y_train, y_test = train_test_split(xFeatures, yLabels, test_size=0.1, random_state=42)

# Feature engineering
tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased')
train_inputs = tokenizer.batch_encode_plus(
    x_train.tolist(),
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
test_inputs = tokenizer.batch_encode_plus(
    x_test.tolist(),
    max_length=512,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)
train_input_ids = train_inputs['input_ids']
train_attention_mask = train_inputs['attention_mask']
test_input_ids = test_inputs['input_ids']
test_attention_mask = test_inputs['attention_mask']
train_labels = torch.tensor(y_train.values, dtype=torch.float)
test_labels = torch.tensor(y_test.values, dtype=torch.float)

# Configuración del modelo BERT
model = BertForSequenceClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-uncased', num_labels=4)

# Entrenamiento del modelo BERT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
train_input_ids = train_input_ids.to(device)
train_attention_mask = train_attention_mask.to(device)
train_labels = train_labels.to(device)
test_input_ids = test_input_ids.to(device)
test_attention_mask = test_attention_mask.to(device)
test_labels = test_labels.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 5
batch_size = 8

for epoch in range(num_epochs):
    model.train()
    train_losses = []
    train_predictions = []
    for i in range(0, len(train_input_ids), batch_size):
        optimizer.zero_grad()
        batch_input_ids = train_input_ids[i:i+batch_size]
        batch_attention_mask = train_attention_mask[i:i+batch_size]
        batch_labels = train_labels[i:i+batch_size]
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_predictions.extend(torch.sigmoid(logits).detach().cpu().numpy())
    train_predictions = np.array(train_predictions)

    model.eval()
    test_predictions = []
    with torch.no_grad():
        for i in range(0, len(test_input_ids), batch_size):
            batch_input_ids = test_input_ids[i:i+batch_size]
            batch_attention_mask = test_attention_mask[i:i+batch_size]
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            test_predictions.extend(torch.sigmoid(logits).cpu().numpy())
    test_predictions = np.array(test_predictions)

    train_loss = np.mean(train_losses)
    train_acc = accuracy_score(train_labels.cpu().numpy(), (train_predictions > 0.5).astype(int))
    test_acc = accuracy_score(test_labels.cpu().numpy(), (test_predictions > 0.5).astype(int))
    test_precision_macro = precision_score(test_labels.cpu().numpy(), (test_predictions > 0.5).astype(int), average='macro')
    test_recall_macro = recall_score(test_labels.cpu().numpy(), (test_predictions > 0.5).astype(int), average='macro')
    test_f1_macro = f1_score(test_labels.cpu().numpy(), (test_predictions > 0.5).astype(int), average='macro')

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f}")
    print(f"Test Precision: {test_precision_macro:.4f} | Test Recall: {test_recall_macro:.4f} | Test F1-Score: {test_f1_macro:.4f}")
    print()

# Clasificación con Binary Relevance
#binary_rel_clf = BinaryRelevance(model)
#binary_rel_clf.fit(train_input_ids, train_labels)
#train_pred = binary_rel_clf.predict(train_input_ids)
#test_pred = binary_rel_clf.predict(test_input_ids)
#
#training_accuracy = accuracy_score(train_labels.cpu().numpy(), train_pred.toarray())
#test_accuracy = accuracy_score(test_labels.cpu().numpy(), test_pred.toarray())
#test_precision_macro = precision_score(test_labels.cpu().numpy(), test_pred.toarray(), average='macro')
#test_recall_macro = recall_score(test_labels.cpu().numpy(), test_pred.toarray(), average='macro')
#test_f1_macro = f1_score(test_labels.cpu().numpy(), test_pred.toarray(), average='macro')
#
#print("Binary Relevance - Train Accuracy: {:.4f}".format(training_accuracy))
#print("Binary Relevance - Test Accuracy: {:.4f}".format(test_accuracy))
#print("Binary Relevance - Test Precision: {:.4f}".format(test_precision_macro))
#print("Binary Relevance - Test Recall: {:.4f}".format(test_recall_macro))
#print("Binary Relevance - Test F1-Score: {:.4f}".format(test_f1_macro))


Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuc

Epoch 1/5
Train Loss: 0.2343 | Train Acc: 0.8045 | Test Acc: 0.8900
Test Precision: 0.8728 | Test Recall: 0.7015 | Test F1-Score: 0.7463

Epoch 2/5
Train Loss: 0.1050 | Train Acc: 0.9206 | Test Acc: 0.8833
Test Precision: 0.8110 | Test Recall: 0.8949 | Test F1-Score: 0.8393

Epoch 3/5
Train Loss: 0.0613 | Train Acc: 0.9573 | Test Acc: 0.9033
Test Precision: 0.8456 | Test Recall: 0.8096 | Test F1-Score: 0.8258

Epoch 4/5
Train Loss: 0.0413 | Train Acc: 0.9740 | Test Acc: 0.9133
Test Precision: 0.8594 | Test Recall: 0.8501 | Test F1-Score: 0.8539

Epoch 5/5
Train Loss: 0.0312 | Train Acc: 0.9807 | Test Acc: 0.9100
Test Precision: 0.8221 | Test Recall: 0.8866 | Test F1-Score: 0.8507

